In [1]:
from pyspark.sql import SparkSession
import os
from dotenv import load_dotenv  

In [ ]:
# Update path to match where your JAR file is downloaded - make sure it's absolute path
jdbc_jar_path = "/home/riddhi/Documents/Riddhi-Tech/Projects/Python-Projects/CRUD-Flask/library/postgresql-42.7.3.jar"
spark = SparkSession.builder \
        .appName("PostgreSQLIngestion") \
        .config("spark.jars", jdbc_jar_path) \
        .config("spark.driver.extraClassPath", jdbc_jar_path) \
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/11 17:26:36 WARN Utils: Your hostname, riddhi, resolves to a loopback address: 127.0.1.1; using 192.168.2.38 instead (on interface wlp47s0)
25/07/11 17:26:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/07/11 17:26:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

In [4]:
print(spark.sparkContext._conf.get('spark.jars'))

/home/riddhi/Documents/Riddhi-Tech/Projects/Python-Projects/CRUD-Flask/library/postgresql-42.7.3.jar


In [5]:
# Load environment variables
load_dotenv()

DB_PASSWORD = os.environ.get('DB_PASSWORD')
DB_NAME = os.environ.get('DB_NAME')
DB_USER = os.environ.get('DB_USER')
SCHEMA_NAME = os.environ.get('SCHEMA_NAME')
REGISTRATION_TABLE = os.environ.get('REGISTRATION_TABLE')
print(DB_PASSWORD, DB_NAME, DB_USER, SCHEMA_NAME, REGISTRATION_TABLE)

123abc456 sampledb riddhi_psql users_schema registration


In [6]:

#This sets up the metadata to ingest your source table.
# JDBC URL and properties for PostgreSQL
jdbc_url = "jdbc:postgresql://localhost:5432/"+ DB_NAME
jdbc_properties = {
    "user": DB_USER,
    "password": DB_PASSWORD,
    "driver": "org.postgresql.Driver"
}

table_name = f"{SCHEMA_NAME}.{REGISTRATION_TABLE}"
print(jdbc_url, jdbc_properties, table_name)

jdbc:postgresql://localhost:5432/sampledb {'user': 'riddhi_psql', 'password': '123abc456', 'driver': 'org.postgresql.Driver'} users_schema.registration


In [18]:
# Read data from PostgreSQL into a DataFrame
df = spark.read.jdbc( \
url=jdbc_url, \
table=table_name, \
properties=jdbc_properties \
) 

# Show the schema and first few rows of the DataFrame
df.printSchema()
df.show(1)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- password: string (nullable = true)
 |-- created_at: timestamp (nullable = true)

+---+------------+--------------------+--------------+--------------------+
| id|        name|               email|      password|          created_at|
+---+------------+--------------------+--------------+--------------------+
|  2|Riddhi Patel|riddhi.patel@exam...|SecurePass123!|2025-06-30 22:26:...|
+---+------------+--------------------+--------------+--------------------+
only showing top 1 row


In [19]:
# Write the DataFrame to Parquet format
base_dir = "/home/riddhi/Documents/Riddhi-Tech/Projects/Python-Projects/CRUD-Flask/"
output_dir = os.path.join(base_dir, "datasets", "parquet", "registration_data")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
df.write.parquet(f"{output_dir}/output_data", mode="overwrite")

In [20]:
df2 = spark.read.parquet(f"{output_dir}/output_data")
df2.show(1)

+---+------------+--------------------+--------------+--------------------+
| id|        name|               email|      password|          created_at|
+---+------------+--------------------+--------------+--------------------+
|  2|Riddhi Patel|riddhi.patel@exam...|SecurePass123!|2025-06-30 22:26:...|
+---+------------+--------------------+--------------+--------------------+
only showing top 1 row
